In [1]:
data = !ls data/group_a/SHH*.csv

In [2]:
data

['data/group_a/SHH201409.csv',
 'data/group_a/SHH201410.csv',
 'data/group_a/SHH201411.csv',
 'data/group_a/SHH201412.csv',
 'data/group_a/SHH201501.csv']

# Treatment vs Control

In [3]:
import pandas as pd

In [4]:
# Treatment vs Control
treatment = pd.read_excel("treatment/treatment_groupSH.xlsx")

In [5]:
treatment_code = treatment['code'].tolist()

# Processing

In [6]:
df = pd.DataFrame(columns=pd.read_csv(data[0], nrows=1).columns.tolist())

In [7]:
for i in range(len(data)):
    df_next = pd.read_csv(data[i])
    df_next = df_next.dropna()
    df_next = df_next[(df_next != 0).all(1)]
    df = df.append(df_next, ignore_index=True)

In [8]:
df = df.dropna()

In [9]:
# Shanghai only
df = df[df['stock'].str.contains("SS")]
df = df[~df['stock'].str.contains("A")]
df = df[~df['stock'].str.contains("z")]

In [10]:
df['treatdum'] = df['stock'].apply(lambda x: 1 if int(x[:-3]) in treatment_code else 0)

In [11]:
df['date'] = pd.to_datetime(df['date'])

In [12]:
df.loc[df['date']<=pd.datetime(2014,11,17), 'perioddum'] = 0
df.loc[df['date']>pd.datetime(2014,11,17), 'perioddum'] = 1
df['did_dum'] = df['perioddum']*df['treatdum']

<ipython-input-12-0f47a2e734f0>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.loc[df['date']<=pd.datetime(2014,11,17), 'perioddum'] = 0
<ipython-input-12-0f47a2e734f0>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.loc[df['date']>pd.datetime(2014,11,17), 'perioddum'] = 1


In [13]:
df['quote2trade'] = df['quote_count'] / df['trade_count']

# Processing

In [14]:
df_treatment = df[df['treatdum'] == 1]
df_control = df[df['treatdum'] == 0]

In [15]:
df_control = df_control[df_control["quote2trade"] <= 2]

In [16]:
df_treatment_mean = df_treatment.groupby("date").mean().reset_index()
df_control_mean = df_control.groupby("date").mean().reset_index()

# Visualization

In [19]:
import pandas as pd
from highcharts import Highchart, Highstock
import csv
import datetime

OPTIONS_DEFAULT = {
                  'chart': {'zoomType': 'x'},
                  'navigator': {'enabled': False},
                  'rangeSelector': {'enabled': False},
                  'legend': {'enabled': True,
                             'itemStyle':{'fontWeight':'normal'}},
                  'title': {"text":"SHH Treatment vs Control: Trade Value"}
                    }

options_line = OPTIONS_DEFAULT
options_line['yAxis'] = [{'opposite': False, 'title':{'text': 'Trade Value'},'tickAmount':8 }]

Line_Chart = Highstock()

def plotline(date, text):
    line = {
                'color': '#c8c8c8',
                'width': 1,
                'value': ((date -  datetime.datetime.utcfromtimestamp(0)).total_seconds() * 1000.0), 
                'zIndex': 1,
                'label': {'text': text,
                            'style': {
                            'color': 'grey'},
                            'rotation': 90,
                            'align': 'center',
                            'x': 3,
                            'y': 50}
            }
    return line

event_dict = {'2014-11-17': 'Stock Connect'}

for date in event_dict:
    date_object = datetime.datetime.strptime(date, '%Y-%m-%d')
    line = plotline(date_object, event_dict[date])
    try:
        options_line['xAxis']['plotLines'].append(line)
    except KeyError:
        options_line['xAxis']={'plotLines': [line]}

Line_Chart.set_dict_options(options_line)

Line_Chart.add_data_set(df_treatment_mean.filter(['date','trade_value']).values.tolist(), name='treatment', yAxis=0)
Line_Chart.add_data_set(df_control_mean.filter(['date','trade_value']).values.tolist(), name='control', yAxis=0)

Line_Chart

# DID

In [59]:
import pandas as pd
from linearmodels.panel import PanelOLS

df.loc[df['date-time']<=pd.datetime(2014,11,17), 'perioddum'] = 0
df.loc[df['date-time']>pd.datetime(2014,11,17), 'perioddum'] = 1
df['did_dum'] = df['perioddum']*df['treatdum']

## set index if you need to identify entity_effects/time_effects
df = df.set_index(['date-time'])

## Now run the DID estimation
x_variables = ['perioddum', 'treatdum', 'did_dum']
y_variable = 'trade_volume'

regression=PanelOLS(df[y_variable], df[x_variables], entity_effects=True, drop_absorbed=True)
regression.fit()

<ipython-input-59-3051c18f260a>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.loc[df['date-time']<=pd.datetime(2014,11,17), 'perioddum'] = 0
<ipython-input-59-3051c18f260a>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.loc[df['date-time']>pd.datetime(2014,11,17), 'perioddum'] = 1


ValueError: Series can only be used with a 2-level MultiIndex